In [16]:
import pandas as pd

df_22_23 = pd.read_csv('csv/2022-2023.csv', encoding='cp949')

# Drop unnecessary column
df_22_23 = df_22_23.drop(['Rk'], axis=1)

# df_22_23.head()
# df_22_23.info()

In [17]:
# Gls' 열에서 최대값을 찾습니다:
max_gls = df_22_23['Gls'].max()

players_with_max_gls = df_22_23[df_22_23['Gls'] == max_gls]['Player']
print(players_with_max_gls)

In [2]:
import numpy as np

# 값 대체를 위한 사전 정의
replace_values = {
    'DF,FW': 'FW,DF',
    'DF,MF': 'MF,DF',
    'MF,FW': 'FW,MF'
}

df_22_23['Pos'] = df_22_23['Pos'].replace(replace_values)
# print(np.unique(df_22_23.Pos))

# 결측치 채우기

In [3]:
numeric_df = df_22_23.select_dtypes(include=[np.number])

# NaN 값을 포함하는 각 열의 평균값으로 NaN을 채우고 소수점 아래 세 자리로 반올림
df_22_23[numeric_df.columns] = numeric_df.apply(lambda x: x.fillna(x.mean()), axis=0).round(3)

# # 열 별로 NaN 값이 있는지 확인
# column_nan = df_22_23.isna().any()
# print(column_nan)
# 
# # 행 별로 NaN 값이 있는지 확인
# row_nan = df_22_23.isna().any(axis=1)
# print(row_nan)

# Baseline model

In [4]:
# Decision Tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import category_encoders as ce # object 타입 수치형 데이터로 인코딩

X = df_22_23.drop(columns=['Gls', 'Player'], axis=1) # Player이름을 모델에서 추출하기 위해 학습 시에는 제거
y = df_22_23['Gls']

# 타겟 인코더: 각 변수들(colums)이 골 수와 어떤 상관관계를 가지고 있는지를 모델이 학습할 수 있게 함.
target_encoder = ce.TargetEncoder(cols=['Nation', 'Pos', 'Squad', 'Comp', 'Matches'])

# 인코딩 수행
df_22_23_encoded = target_encoder.fit_transform(X, y)

# 데이터프레임 재구성
df_22_23_encoded['Gls'] = df_22_23['Gls']

X_encoded_train = df_22_23_encoded.drop(columns=['Gls'], axis=1)
y_encoded_train = df_22_23_encoded['Gls']

X_train, X_val, y_train, y_val = train_test_split(
    X_encoded_train, y_encoded_train,
    test_size=0.2,
    random_state=42
)

model = DecisionTreeRegressor(random_state=42, max_features=0.9)
model.fit(X_train, y_train)

coef = model.score(X_val, y_val) # Return the coefficient of determination of the prediction.
print(f'Decision Tree R^2: {coef:.2f} (or {coef * 100:.2f}%)')

# 예측결과 확인

In [ ]:
# 테스트 데이터셋에 대한 예측
predictions = model.predict(X_encoded_train)

# 예측값 중 최대값 찾기
max_prediction_index = predictions.argmax()
max_prediction_player = df_22_23.iloc[max_prediction_index]['Player']

print("예측된 득점왕 선수는:", max_prediction_player)

# 예측 결과 DataFrame 생성
predicted_data = pd.DataFrame({
    'Player': X_encoded_train.index.map(df_22_23['Player']),  # 인덱스를 사용하여 선수 이름 매핑
    'Predicted_Gls': predictions
})

# 실제 득점왕
actual_top_scorer = df_22_23.loc[df_22_23['Gls'].idxmax(), 'Player']

# 예측 득점왕
predicted_top_scorer = predicted_data.loc[predicted_data['Predicted_Gls'].idxmax(), 'Player']

# 득점왕 비교
if actual_top_scorer == predicted_top_scorer:
    print(f"모델이 정확하게 득점왕을 예측했습니다: {actual_top_scorer}")
else:
    print(f"모델의 예측 득점왕: {predicted_top_scorer}, 실제 득점왕: {actual_top_scorer}")

# idx = df_22_23.index[df_22_23['Player'] == 'Erling Haaland']
# df_22_23.loc[idx, 'Gls']

# Improve accuracy
    - RandomForestRegressor: 정확한 예측을 만들기 위한 각 특성의 상대적인 중요도를 알려준다.

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

random_forest = RandomForestRegressor(n_jobs=-1)
# 
rf_params = {
    'random_state': [42],
    'n_estimators': [100, 120, 140],
    'max_depth': [None, 3, 5, 7, 10],
    'max_features': [0.5, 0.7, 0.9, 1.0]
}

gs = GridSearchCV(
    estimator=random_forest,
    param_grid=rf_params,
    scoring='neg_mean_absolute_error', # 사이킷런의 교차검증은 클수록 좋은 효용함수를 기대
    cv=5
)

gs.fit(X_train, y_train)
print(f'Best parameters: {gs.best_params_}')

# 분할 전 데이터셋에 대한 예측
preds = gs.best_estimator_.predict(X_encoded_train) # 오차가 섞였다고 생각할 수 있다
score = mean_absolute_error(y_encoded_train, preds)

print(f'Random Forest evaluation by MAE: {score:.2f}')

# 예측값 중 최대값 찾기
max_prediction_index = preds.argmax()
max_prediction_player = df_22_23.iloc[max_prediction_index]['Player']

print("예측된 득점왕 선수는:", max_prediction_player)

# Apply StratifiedKFold & Nested CV
    - StratifiedKFold는 각 폴드의 클래스 분포가 원래 데이터의 클래스 분포와 유사하게 유지되도록 데이터를 나눈다.
    - StratifiedKFold의 split 메서드는 입력된 데이터와 목표 변수의 분포를 기반으로 폴드를 생성
        - 각 폴드의 훈련 및 검증 인덱스를 생성

In [15]:
from sklearn.model_selection import StratifiedKFold

# StratifiedKFold가 클래스 분포를 유지하며 폴드를 나누는 데 사용
# 득점 데이터를 분포를 고려하여 범주화하여 StratifiedKFold 적용
y_binned = pd.qcut(y_encoded_train, q=5, labels=False, duplicates='drop')

inner_cv = StratifiedKFold(
    n_splits=3,
    random_state=42,
    shuffle=True)
# outer_cv = StratifiedKFold(
#     n_splits=5,
#     random_state=42,
#     shuffle=True
# )
# # 
# # Nested Cross Validation
# outer_scores = []
# for train_index, valid_index in outer_cv.split(X_encoded_train, y_binned): # train_index, valid_index: ndarray
#     X_train, X_valid = X_encoded_train.iloc[train_index], X_encoded_train.iloc[valid_index]
#     y_train, y_valid = y_encoded_train.iloc[train_index], y_encoded_train.iloc[valid_index]
#     y_binned_train = y_binned.iloc[train_index]
# 
#     # inner loop: Tune hyperparameters
#     gs = GridSearchCV(
#         estimator=random_forest,
#         param_grid=rf_params,
#         scoring='neg_mean_absolute_error',
#         cv=inner_cv.split(X_train, y_binned_train) # 튜플 반환: 각 폴드마다 어떤 데이터를 훈련에 사용하고, 어떤 데이터를 검증에 사용할지를 명확하게 정해준다
#     )
#     gs.fit(X_train, y_train)
# 
#     predictions = gs.best_estimator_.predict(X_valid)
#     outer_score = mean_absolute_error(y_valid, predictions)
#     outer_scores.append(outer_score)
# 
# average_score = np.mean(outer_scores)  # 평균 점수 계산
# print(f'Average MAE across all folds: {average_score:0.3f}')

# 전체 데이터를 사용하여 최종 모델 훈련
final_gs = GridSearchCV(
    estimator=random_forest,
    param_grid=rf_params,
    scoring='neg_mean_absolute_error',
    cv=inner_cv.split(X_encoded_train, y_binned)
)

final_gs.fit(X_encoded_train, y_encoded_train)

# print(f"X_encoded_train의 형태: {X_encoded_train.shape}") # (2889, 24)
# print(f"y_encoded_train의 형태: {y_encoded_train.shape}") # (2889,)
# print(f"y_binned의 형태: {y_binned.shape}") # (2889,)

# 전체 데이터로 예측 수행
preds = final_gs.best_estimator_.predict(X_encoded_train)
MAE = mean_absolute_error(y_encoded_train, preds)
print(f'Random Forest evaluation MAE: {MAE:.2f}')

# 예측된 득점왕 선수 찾기
max_prediction_index = preds.argmax()
max_prediction_player = df_22_23.iloc[max_prediction_index]['Player']

print("예측된 득점왕 선수는:", max_prediction_player)

# Test Data

In [ ]:
import pandas as pd

df_23_24 = pd.read_csv('csv/2023-2024.csv', encoding='cp949')

# Drop unnecessary column
df_23_24 = df_23_24.drop(['Rk'], axis=1)

# 'Age' 열에서 '-' 앞의 정보만 추출하여 새로운 열로 저장
df_23_24['Age'] = df_23_24['Age'].str.split('-').str[0]

# # 'Age'열 데이터 타입 변경
# df_23_24['Age'] = df_23_24['Age'].astype('Int64')

# 숫자로 변환할 수 없는 값이 있는 경우 NaN으로 변경(object -> NaN)
df_23_24['Age'] = pd.to_numeric(df_23_24['Age'], errors='coerce')

# df_23_24['Age'] = df_23_24['Age'].astype(int)
# IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

# df_23_24.head()

In [ ]:
replace_values = {
    'DF,FW': 'FW,DF',
    'DF,MF': 'MF,DF',
    'MF,FW': 'FW,MF'
}

df_23_24['Pos'] = df_23_24['Pos'].replace(replace_values)

In [ ]:
numeric_df = df_23_24.select_dtypes(include=[np.number])

df_23_24[numeric_df.columns] = numeric_df.apply(lambda x: x.fillna(x.mean()), axis=0).round(2)

# 'Age' 열의 타입을 'int64'로 변경
df_23_24['Age'] = df_23_24['Age'].astype(int)

# # 열 별로 NaN 값이 있는지 확인
# column_nan = df_23_24.isna().any()
# print(column_nan)

# 독립변수, 종속변수 생성

In [ ]:
X_test = df_23_24.drop(columns=['Gls', 'Player'], axis=1)
y_test = df_23_24['Gls']

target_encoder = ce.TargetEncoder(cols=['Nation', 'Pos', 'Squad', 'Comp', 'Matches'])

# 인코딩 수행
df_23_24_encoded = target_encoder.fit_transform(X_test, y_test)

# 데이터프레임 재구성
df_23_24_encoded['Gls'] = df_23_24['Gls']

X_encoded_test = df_23_24_encoded.drop(columns=['Gls'], axis=1)
y_encoded_test = df_23_24_encoded['Gls']

# Test Data로 예측하기

In [ ]:
# print(f"X_encoded_test의 형태: {X_encoded_test.shape}") # shape: (2793, 24)
# print(f"y_encoded_test의 형태: {y_encoded_test.shape}") # shape: (2793,)

final_gs.best_estimator_.fit(X_encoded_test, y_encoded_test)

preds_test = final_gs.best_estimator_.predict(X_encoded_test)
score = mean_absolute_error(y_encoded_test, preds_test)

print(f'Random Forest evaluation by MAE: {score:.2f}')

# 예측값 중 최대값 찾기
max_prediction_index = preds_test.argmax()
max_prediction_player = df_23_24.iloc[max_prediction_index]['Player']

print("예측된 득점왕 선수는:", max_prediction_player)